# Big Data Assignment (LinearRegression)
## Topic :- LinearRegression
##submitted To:- Bharti Motwani Ma'am
##Submitted By:- Yash Rathore(MSc Data Science and Forcasting 2nd semester)

In [4]:
pip install pyspark


In [5]:
from pyspark.sql import SparkSession
session=SparkSession.builder.appName("/content/Health_insurance.csv").getOrCreate()

In [6]:
df=session.read.csv("Health_insurance.csv", header=True, inferSchema=True)

In [7]:
df.count()

1338

In [8]:
len(df.columns)

7

In [9]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- sex: string (nullable = true)
 |-- bmi: double (nullable = true)
 |-- children: integer (nullable = true)
 |-- smoker: string (nullable = true)
 |-- region: string (nullable = true)
 |-- charges: double (nullable = true)



In [10]:
df.describe().show()

+-------+------------------+------+------------------+-----------------+------+---------+------------------+
|summary|               age|   sex|               bmi|         children|smoker|   region|           charges|
+-------+------------------+------+------------------+-----------------+------+---------+------------------+
|  count|              1338|  1338|              1338|             1338|  1338|     1338|              1338|
|   mean| 39.20702541106129|  null|30.663396860986538|  1.0949177877429|  null|     null|13270.422265141257|
| stddev|14.049960379216147|  null| 6.098186911679012|1.205492739781914|  null|     null|12110.011236693992|
|    min|                18|female|             15.96|                0|    no|northeast|         1121.8739|
|    max|                64|  male|             53.13|                5|   yes|southwest|       63770.42801|
+-------+------------------+------+------------------+-----------------+------+---------+------------------+



In [11]:
df.head()

Row(age=19, sex='female', bmi=27.9, children=0, smoker='yes', region='southwest', charges=16884.924)

In [12]:
from pyspark.ml.feature import StringIndexer

In [13]:
indexer = StringIndexer(inputCol="sex",outputCol="sex_cat")
Indexed = indexer.fit(df).transform(df)

In [14]:
indexer = StringIndexer(inputCol="smoker",outputCol="smoker_cat")
Indexed = indexer.fit(Indexed).transform(Indexed)

In [15]:
indexer = StringIndexer(inputCol="region",outputCol="region_cat")
Indexed = indexer.fit(Indexed).transform(Indexed)

In [16]:
Indexed.show()

+---+------+------+--------+------+---------+-----------+-------+----------+----------+
|age|   sex|   bmi|children|smoker|   region|    charges|sex_cat|smoker_cat|region_cat|
+---+------+------+--------+------+---------+-----------+-------+----------+----------+
| 19|female|  27.9|       0|   yes|southwest|  16884.924|    1.0|       1.0|       2.0|
| 18|  male| 33.77|       1|    no|southeast|  1725.5523|    0.0|       0.0|       0.0|
| 28|  male|  33.0|       3|    no|southeast|   4449.462|    0.0|       0.0|       0.0|
| 33|  male|22.705|       0|    no|northwest|21984.47061|    0.0|       0.0|       1.0|
| 32|  male| 28.88|       0|    no|northwest|  3866.8552|    0.0|       0.0|       1.0|
| 31|female| 25.74|       0|    no|southeast|  3756.6216|    1.0|       0.0|       0.0|
| 46|female| 33.44|       1|    no|southeast|  8240.5896|    1.0|       0.0|       0.0|
| 37|female| 27.74|       3|    no|northwest|  7281.5056|    1.0|       0.0|       1.0|
| 37|  male| 29.83|       2|    

In [17]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [18]:
Indexed.columns

['age',
 'sex',
 'bmi',
 'children',
 'smoker',
 'region',
 'charges',
 'sex_cat',
 'smoker_cat',
 'region_cat']

In [19]:
assembler = VectorAssembler(inputCols=["age","bmi","children","sex_cat",
                                       "smoker_cat","region_cat"],
                            outputCol="Features")


In [20]:
assembler

VectorAssembler_9e68cf9b8160

In [21]:
output = assembler.transform(Indexed)

In [22]:
output.show()

+---+------+------+--------+------+---------+-----------+-------+----------+----------+--------------------+
|age|   sex|   bmi|children|smoker|   region|    charges|sex_cat|smoker_cat|region_cat|            Features|
+---+------+------+--------+------+---------+-----------+-------+----------+----------+--------------------+
| 19|female|  27.9|       0|   yes|southwest|  16884.924|    1.0|       1.0|       2.0|[19.0,27.9,0.0,1....|
| 18|  male| 33.77|       1|    no|southeast|  1725.5523|    0.0|       0.0|       0.0|[18.0,33.77,1.0,0...|
| 28|  male|  33.0|       3|    no|southeast|   4449.462|    0.0|       0.0|       0.0|[28.0,33.0,3.0,0....|
| 33|  male|22.705|       0|    no|northwest|21984.47061|    0.0|       0.0|       1.0|[33.0,22.705,0.0,...|
| 32|  male| 28.88|       0|    no|northwest|  3866.8552|    0.0|       0.0|       1.0|[32.0,28.88,0.0,0...|
| 31|female| 25.74|       0|    no|southeast|  3756.6216|    1.0|       0.0|       0.0|[31.0,25.74,0.0,1...|
| 46|female| 33.44|

In [23]:
output.select("Features","charges").show(truncate=False)

+-----------------------------+-----------+
|Features                     |charges    |
+-----------------------------+-----------+
|[19.0,27.9,0.0,1.0,1.0,2.0]  |16884.924  |
|[18.0,33.77,1.0,0.0,0.0,0.0] |1725.5523  |
|[28.0,33.0,3.0,0.0,0.0,0.0]  |4449.462   |
|[33.0,22.705,0.0,0.0,0.0,1.0]|21984.47061|
|[32.0,28.88,0.0,0.0,0.0,1.0] |3866.8552  |
|[31.0,25.74,0.0,1.0,0.0,0.0] |3756.6216  |
|[46.0,33.44,1.0,1.0,0.0,0.0] |8240.5896  |
|[37.0,27.74,3.0,1.0,0.0,1.0] |7281.5056  |
|[37.0,29.83,2.0,0.0,0.0,3.0] |6406.4107  |
|[60.0,25.84,0.0,1.0,0.0,1.0] |28923.13692|
|[25.0,26.22,0.0,0.0,0.0,3.0] |2721.3208  |
|[62.0,26.29,0.0,1.0,1.0,0.0] |27808.7251 |
|[23.0,34.4,0.0,0.0,0.0,2.0]  |1826.843   |
|[56.0,39.82,0.0,1.0,0.0,0.0] |11090.7178 |
|[27.0,42.13,0.0,0.0,1.0,0.0] |39611.7577 |
|[19.0,24.6,1.0,0.0,0.0,2.0]  |1837.237   |
|[52.0,30.78,1.0,1.0,0.0,3.0] |10797.3362 |
|[23.0,23.845,0.0,0.0,0.0,3.0]|2395.17155 |
|[56.0,40.3,0.0,0.0,0.0,2.0]  |10602.385  |
|[30.0,35.3,0.0,0.0,1.0,2.0]  |3

In [24]:
final_data =output.select("Features","charges")

In [25]:
final_data.show()

+--------------------+-----------+
|            Features|    charges|
+--------------------+-----------+
|[19.0,27.9,0.0,1....|  16884.924|
|[18.0,33.77,1.0,0...|  1725.5523|
|[28.0,33.0,3.0,0....|   4449.462|
|[33.0,22.705,0.0,...|21984.47061|
|[32.0,28.88,0.0,0...|  3866.8552|
|[31.0,25.74,0.0,1...|  3756.6216|
|[46.0,33.44,1.0,1...|  8240.5896|
|[37.0,27.74,3.0,1...|  7281.5056|
|[37.0,29.83,2.0,0...|  6406.4107|
|[60.0,25.84,0.0,1...|28923.13692|
|[25.0,26.22,0.0,0...|  2721.3208|
|[62.0,26.29,0.0,1...| 27808.7251|
|[23.0,34.4,0.0,0....|   1826.843|
|[56.0,39.82,0.0,1...| 11090.7178|
|[27.0,42.13,0.0,0...| 39611.7577|
|[19.0,24.6,1.0,0....|   1837.237|
|[52.0,30.78,1.0,1...| 10797.3362|
|[23.0,23.845,0.0,...| 2395.17155|
|[56.0,40.3,0.0,0....|  10602.385|
|[30.0,35.3,0.0,0....|  36837.467|
+--------------------+-----------+
only showing top 20 rows



In [26]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [27]:
train_data.show()

+--------------------+-----------+
|            Features|    charges|
+--------------------+-----------+
|(6,[0,1],[18.0,30...|  1131.5066|
|(6,[0,1],[18.0,33...|  1135.9407|
|(6,[0,1],[18.0,33...|  1136.3994|
|(6,[0,1],[18.0,37...|  1141.4451|
|(6,[0,1],[18.0,41...|  1146.7966|
|(6,[0,1],[18.0,43...|  1149.3959|
|(6,[0,1],[18.0,53...|  1163.4627|
|(6,[0,1],[20.0,33...|  1391.5287|
|(6,[0,1],[21.0,23...|  1515.3449|
|(6,[0,1],[21.0,31...|16586.49771|
|(6,[0,1],[21.0,35...|  1532.4697|
|(6,[0,1],[21.0,36...|  1534.3045|
|(6,[0,1],[22.0,26...|  1664.9996|
|(6,[0,1],[22.0,33...|  1674.6323|
|(6,[0,1],[23.0,41...|  1837.2819|
|(6,[0,1],[24.0,35...|  1986.9334|
|(6,[0,1],[27.0,23...|   2483.736|
|(6,[0,1],[27.0,32...|  2497.0383|
|(6,[0,1],[27.0,33...|  2498.4144|
|(6,[0,1],[28.0,38...|  2689.4954|
+--------------------+-----------+
only showing top 20 rows



In [28]:
test_data.show()

+--------------------+-----------+
|            Features|    charges|
+--------------------+-----------+
|(6,[0,1],[18.0,23...|  1121.8739|
|(6,[0,1],[18.0,34...|   1137.011|
|(6,[0,1],[18.0,34...|  1137.4697|
|(6,[0,1],[23.0,26...|  1815.8759|
|(6,[0,1],[23.0,32...|  1824.2854|
|(6,[0,1],[24.0,32...|  1981.5819|
|(6,[0,1],[25.0,25...|  2137.6536|
|(6,[0,1],[26.0,35...|  2322.6218|
|(6,[0,1],[34.0,34...|  3935.1799|
|(6,[0,1],[37.0,36...|19214.70553|
|(6,[0,1],[41.0,40...|  5709.1644|
|(6,[0,1],[48.0,40...|  7804.1605|
|(6,[0,1],[53.0,41...|  9504.3103|
|(6,[0,1],[56.0,34...| 10594.2257|
|(6,[0,1],[62.0,38...| 12981.3457|
|(6,[0,1],[62.0,39...| 12982.8747|
|[18.0,21.47,0.0,0...|  1702.4553|
|[18.0,21.66,0.0,1...| 14283.4594|
|[18.0,22.99,0.0,0...|  1704.5681|
|[18.0,23.32,1.0,0...|  1711.0268|
+--------------------+-----------+
only showing top 20 rows



In [29]:
from pyspark.ml.regression import LinearRegression

In [30]:
lr = LinearRegression(featuresCol="Features",
                      labelCol="charges")

In [31]:
trained_model = lr.fit(train_data)

In [32]:
trained_model

LinearRegressionModel: uid=LinearRegression_6283938a428d, numFeatures=6

In [33]:
results = trained_model.evaluate(train_data)

In [34]:
print(results.r2)

0.7337589628109955


In [36]:
print(results.meanSquaredError)

38573552.693008214


In [37]:
print(results.meanAbsoluteError)

4274.813489231799


In [38]:
unlabeled_data = test_data.select("Features")

In [39]:
unlabeled_data.show()

+--------------------+
|            Features|
+--------------------+
|(6,[0,1],[18.0,23...|
|(6,[0,1],[18.0,34...|
|(6,[0,1],[18.0,34...|
|(6,[0,1],[23.0,26...|
|(6,[0,1],[23.0,32...|
|(6,[0,1],[24.0,32...|
|(6,[0,1],[25.0,25...|
|(6,[0,1],[26.0,35...|
|(6,[0,1],[34.0,34...|
|(6,[0,1],[37.0,36...|
|(6,[0,1],[41.0,40...|
|(6,[0,1],[48.0,40...|
|(6,[0,1],[53.0,41...|
|(6,[0,1],[56.0,34...|
|(6,[0,1],[62.0,38...|
|(6,[0,1],[62.0,39...|
|[18.0,21.47,0.0,0...|
|[18.0,21.66,0.0,1...|
|[18.0,22.99,0.0,0...|
|[18.0,23.32,1.0,0...|
+--------------------+
only showing top 20 rows



In [40]:
predictions = trained_model.transform(unlabeled_data)

In [41]:
predictions.show()

+--------------------+-------------------+
|            Features|         prediction|
+--------------------+-------------------+
|(6,[0,1],[18.0,23...|-338.72404738003934|
|(6,[0,1],[18.0,34...| 3073.9339084803014|
|(6,[0,1],[18.0,34...|  3177.347785930613|
|(6,[0,1],[23.0,26...| 2000.2606504236046|
|(6,[0,1],[23.0,32...|  3896.181737012683|
|(6,[0,1],[24.0,32...| 3984.7944592555923|
|(6,[0,1],[25.0,25...|  2280.899972359739|
|(6,[0,1],[26.0,35...|  5575.342895562369|
|(6,[0,1],[34.0,34...|  7283.912155525368|
|(6,[0,1],[37.0,36...|   8687.30297420755|
|(6,[0,1],[41.0,40...| 11006.617534735155|
|(6,[0,1],[48.0,40...| 12798.930534872428|
|(6,[0,1],[53.0,41...| 14517.431967974191|
|(6,[0,1],[56.0,34...| 13094.176803014476|
|(6,[0,1],[62.0,38...| 16038.843610312602|
|(6,[0,1],[62.0,39...|  16383.55653514698|
|[18.0,21.47,0.0,0...|  -561.414114539466|
|[18.0,21.66,0.0,1...|  23662.85935808099|
|[18.0,22.99,0.0,0...|  -85.0835274955989|
|[18.0,23.32,1.0,0...| 302.56733335104036|
+----------